<a href="https://colab.research.google.com/github/usshaa/SMBDA/blob/main/C-5.10%3A%20Predicting_Customer_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Building Machine Learning Pipelines in PySpark

Machine learning pipelines in PySpark streamline the process of building and deploying machine learning models by chaining multiple stages, including data preprocessing, feature engineering, model training, and evaluation, into a cohesive workflow. Here's a detailed guide on building a machine learning pipeline using PySpark.

### Example: Predicting Customer Churn

We'll build a machine learning pipeline for a customer churn prediction task using a hypothetical dataset with columns: `User Id`, `First Name`, `Last Name`, `Sex`, `Email`, `Phone`, `Date of birth`, `Job Title`, and a binary label `Churn`.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff, current_date
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from faker import Faker
from random import choice
import pandas as pd

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4125139553725403>:7
      5 from pyspark.ml import Pipeline
      6 from pyspark.ml.evaluation import BinaryClassificationEvaluator
----> 7 from faker import Faker
      8 from random import choice
      9 import pandas as pd

File /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:171, in _create_import_patch.<locals>.import_patch(name, globals, locals, fromlist, level)
    166 thread_local._nest_level += 1
    168 try:
    169     # Import the desired module. If you’re seeing this while debugging a failed import,
    170     # look at preceding stack frames for relevant error information.
--> 171     original_result = python_builtin_import(name, globals, locals, fromlist, level)
    173     is_root_import = thread_local._nest_level == 1
    174     # `level` represents the number of leadin

In [ ]:
# Step 1: Initialize Spark Session
spark = SparkSession.builder \
    .appName("ML Pipeline Example") \
    .getOrCreate()

In [ ]:
# Step 2: Load Data
data = spark.read.csv("/FileStore/tables/customer_data.csv", header=True, inferSchema=True)

In [ ]:
# Step 3: Data Preprocessing
indexer_sex = StringIndexer(inputCol="Sex", outputCol="SexIndex")
indexer_job = StringIndexer(inputCol="Job Title", outputCol="JobTitleIndex", handleInvalid="keep")

In [ ]:
# Calculate Age from Date of Birth
data = data.withColumn("Age", (datediff(current_date(), col("Date of birth")) / 365).cast("int"))

In [ ]:
# Select relevant features for the model
assembler = VectorAssembler(
    inputCols=["Age", "SexIndex", "JobTitleIndex"],
    outputCol="features"
)

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

In [ ]:
# Step 4: Build Machine Learning Pipeline
lr = LogisticRegression(featuresCol="scaledFeatures", labelCol="Churn")
pipeline = Pipeline(stages=[indexer_sex, indexer_job, assembler, scaler, lr])

In [ ]:
# Step 5: Train and Evaluate the Model
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

In [ ]:
model = pipeline.fit(train_data)
predictions = model.transform(test_data)

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="Churn")

In [ ]:
accuracy = evaluator.evaluate(predictions)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.54


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Create a MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="Churn", predictionCol="prediction", metricName="weightedPrecision")
precision = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol="Churn", predictionCol="prediction", metricName="weightedRecall")
recall = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(labelCol="Churn", predictionCol="prediction", metricName="f1")
f1_score = evaluator.evaluate(predictions)

# Print the evaluation metrics
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1_score:.2f}")

Precision: 0.58
Recall: 0.56
F1-score: 0.53


In [ ]:
from pyspark.sql import functions as F
import pandas as pd

# Assuming 'predictions' is your DataFrame containing predictions
predictions = predictions.withColumn("prediction", F.col("prediction").cast("double"))
predictions = predictions.withColumn("Churn", F.col("Churn").cast("double"))

# Group by true and predicted labels and count occurrences
conf_matrix = predictions.groupBy("Churn", "prediction").count()

# Convert to Pandas DataFrame for easier manipulation
conf_matrix_pd = conf_matrix.toPandas()

# Pivot the DataFrame to get confusion matrix
conf_matrix_pd = conf_matrix_pd.pivot(index='Churn', columns='prediction', values='count').fillna(0)

# Display confusion matrix
print("Confusion Matrix:")
conf_matrix_pd

Confusion Matrix:


prediction,0.0,1.0
Churn,,
0.0,26,56
1.0,16,64


In [ ]:
# Step 6: Save and Load Model (Optional)
model.save("pipeline_model")